In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re 
import seaborn as sns
from collections import ChainMap
from collections import Counter
from datetime import datetime
from itertools import combinations
from scipy.stats import ttest_ind
from sklearn import metrics # инструменты для оценки точности модели  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn.model_selection import train_test_split  # Загружаем специальный инструмент для разбивки: 
from statsmodels.stats import weightstats

pd.set_option('display.max_rows', 50)  # показывать больше строк
pd.set_option('display.max_columns', 50)  # показывать больше колонок

In [2]:
main_task = pd.read_csv('main_task.csv')


In [3]:
#  предобработка
# для удобства переименуем переменные
main_task.columns = ['restaurant_id', 'city', 'cuisine_style', 'ranking', 'rating',
              'price_range', 'number_of_reviews', 'reviews', 'URL_TA', 'ID_TA']

df = main_task.copy()



*  Restaurant_id — идентификационный номер ресторана / сети ресторанов;
*   City — город, в котором находится ресторан;
*  Cuisine Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
*  Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города;
* Rating — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
*    Price Range — диапазон цен в ресторане;
*   Number of Reviews — количество отзывов о ресторане;
*  Reviews — данные о двух отзывах, которые отображаются на сайте ресторана;
* URL_TA — URL страницы ресторана на TripAdvisor;
* ID_TA — идентификатор ресторана в базе данных TripAdvisor.


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   restaurant_id      40000 non-null  object 
 1   city               40000 non-null  object 
 2   cuisine_style      30717 non-null  object 
 3   ranking            40000 non-null  float64
 4   rating             40000 non-null  float64
 5   price_range        26114 non-null  object 
 6   number_of_reviews  37457 non-null  float64
 7   reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [5]:
df.sample(5)

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,URL_TA,ID_TA
32217,id_460,Warsaw,"['Italian', 'Mediterranean']",461.0,4.5,$$ - $$$,23.0,"[['Amazing', 'Ignore the other reviews.'], ['0...",/Restaurant_Review-g274856-d10429587-Reviews-P...,d10429587
39782,id_7485,London,"['Wine Bar', 'British']",7494.0,4.0,$$ - $$$,19.0,"[['Brokers Wine Bar', 'Great little place in t...",/Restaurant_Review-g186338-d6834397-Reviews-Br...,d6834397
26663,id_1040,Hamburg,NaN,1042.0,4.5,NaN,9.0,[['One of the best breakfast places in Hambur....,/Restaurant_Review-g187331-d12096070-Reviews-B...,d12096070
27516,id_3192,Berlin,NaN,3194.0,5.0,NaN,4.0,"[['Excellent food'], ['01/03/2017']]",/Restaurant_Review-g187323-d10860258-Reviews-B...,d10860258
13008,id_4435,London,"['Italian', 'Pizza', 'Vegetarian Friendly', 'G...",4442.0,4.0,$$ - $$$,225.0,"[['Great pizza from a wood-fired oven', 'Excel...",/Restaurant_Review-g186338-d720402-Reviews-Red...,d720402


In [6]:
type(df.loc[0,'reviews'])
# data.loc[0,'reviews']

str

На первом этапе для создания такого датафрейма давайте просто удалим столбцы, содержащие данные типа __object__, и заполним пропущенные значения (__None__ или __NaN__) каким-то одним значением (нулём или средним арифметическим) для всего столбца.

In [7]:
df_1 = df.copy()
display(df_1.info())
df_1[df_1.number_of_reviews.isna()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   restaurant_id      40000 non-null  object 
 1   city               40000 non-null  object 
 2   cuisine_style      30717 non-null  object 
 3   ranking            40000 non-null  float64
 4   rating             40000 non-null  float64
 5   price_range        26114 non-null  object 
 6   number_of_reviews  37457 non-null  float64
 7   reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


None

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,URL_TA,ID_TA
8,id_2690,Vienna,NaN,2692.0,4.0,NaN,NaN,"[[], []]",/Restaurant_Review-g190454-d12845029-Reviews-G...,d12845029
21,id_5844,Madrid,NaN,5847.0,4.0,NaN,NaN,"[[], []]",/Restaurant_Review-g187514-d10058810-Reviews-B...,d10058810
32,id_1327,Budapest,NaN,1328.0,5.0,NaN,NaN,"[['Absolutely amazing, tasty, fresh, cheap Ch....",/Restaurant_Review-g274887-d8791087-Reviews-Bu...,d8791087
102,id_1409,Budapest,"['French', 'European', 'Hungarian']",1410.0,5.0,NaN,NaN,"[[], []]",/Restaurant_Review-g274887-d13197631-Reviews-L...,d13197631
108,id_2047,Prague,"['Italian', 'Mediterranean']",2050.0,5.0,$$ - $$$,NaN,"[[], []]",/Restaurant_Review-g274707-d12243659-Reviews-T...,d12243659
...,...,...,...,...,...,...,...,...,...,...
39965,id_5720,Rome,NaN,5722.0,5.0,NaN,NaN,"[[], []]",/Restaurant_Review-g187791-d793376-Reviews-Wes...,d793376
39972,id_2009,Prague,"['Cafe', 'Delicatessen', 'Healthy', 'Wine Bar'...",2012.0,5.0,$,NaN,"[['Personal expirence'], ['10/27/2017']]",/Restaurant_Review-g274707-d12981295-Reviews-M...,d12981295
39986,id_7274,Madrid,NaN,7279.0,3.0,NaN,NaN,"[[], []]",/Restaurant_Review-g187514-d12182212-Reviews-C...,d12182212
39988,id_4968,Berlin,NaN,4970.0,4.0,NaN,NaN,"[[], []]",/Restaurant_Review-g187323-d7761701-Reviews-Ar...,d7761701


In [8]:
#  удалим все нечисловые колонки

df_1 = df_1.drop([column  for column in df_1.columns if df_1[column].dtype == 'O'],axis=1)
df_1.number_of_reviews = df_1.number_of_reviews.fillna(0)
display(df_1.info())
df[df_1.number_of_reviews.isna()]  # выведем все нулевые значения

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ranking            40000 non-null  float64
 1   rating             40000 non-null  float64
 2   number_of_reviews  40000 non-null  float64
dtypes: float64(3)
memory usage: 937.6 KB


None

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,URL_TA,ID_TA


нулевых значений нет

In [9]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
    # Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)  
# X = df.drop(['restaurant_id', 'rating'], axis = 1)  
X = df_1.drop(['rating'], axis = 1)  

y = df_1['rating']  
      
    # Загружаем специальный инструмент для разбивки:  убрано наверх
#     from sklearn.model_selection import train_test_split   
    
      
    # Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
    # Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    

In [10]:
# Импортируем необходимые библиотеки: убрано наверх 
#     from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
#     from sklearn import metrics # инструменты для оценки точности модели  
      
    # Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
    # Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
    # Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.  
    # Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  

In [11]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются  
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.  
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))  

MAE: 0.4268390017857143


 4.1 Вопросы о ценах

Сколько вариантов непустых значений встречается в столбце Price Range?

In [12]:
df_4 = main_task.copy()

In [13]:
variants = [element for element in(df.price_range.unique()) if type(element) == str]
display(len(variants),variants)

3

['$$ - $$$', '$$$$', '$']

In [14]:
df.sample()

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,URL_TA,ID_TA
10400,id_3001,Vienna,['Asian'],3003.0,3.5,NaN,5.0,"[['What Chinese place does not serve rice.', '...",/Restaurant_Review-g190454-d3648199-Reviews-Fe...,d3648199


Сколько ресторанов относятся к среднему ценовому сегменту?

In [15]:
len(df[df.price_range == '$$ - $$$'])

18412

 4.2 Вопрос о городах  
1 point possible (graded)  
Сколько городов представлено в наборе данных?

In [16]:
len(df.city.unique())

31

## 4.3 Вопросы о кухнях

Сколько типов кухонь представлено в наборе данных?

In [17]:
df = main_task.copy()
df.cuisine_style = df.cuisine_style.str.replace("'", '')
df.cuisine_style = df.cuisine_style.str.replace("[", '')
df.cuisine_style = df.cuisine_style.str.replace("]", '')
df.cuisine_style = df.cuisine_style.str.replace(" ", '')
df.cuisine_style = df.cuisine_style.str.split(',')
df = df.explode('cuisine_style')
df.head()

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,URL_TA,ID_TA
0,id_5569,Paris,European,5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
0,id_5569,Paris,French,5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
0,id_5569,Paris,International,5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,Japanese,353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781


In [18]:
# не работает, пока что
df_4 = main_task.copy()

# df_4.cuisine_style = df_4.cuisine_style.str.replace(("'","[","]"," "), '')
df_4.cuisine_style = df_4.cuisine_style.str.strip("'' []")

# df_4.cuisine_style = df_4.cuisine_style.str.strip()
# df_4.cuisine_style = df_4.cuisine_style.str.split(',')
# df_4 = df_4.explode('cuisine_style')
display(type(df_4.cuisine_style.loc[0]))
df_4.cuisine_style.loc[0]

str

"European', 'French', 'International"

In [19]:
cuisine = [element for element in (df.cuisine_style.unique()) if type(element) == str]
display(len(cuisine))
df.cuisine_style.value_counts()

125

VegetarianFriendly    11189
European              10060
Mediterranean          6277
Italian                5964
VeganOptions           4486
                      ...  
Yunnan                    1
Xinjiang                  1
Salvadoran                1
Burmese                   1
Latvian                   1
Name: cuisine_style, Length: 125, dtype: int64

Какое среднее количество кухонь предлагается в одном ресторане? Если в данных отсутствует информация о типах кухонь, то считайте, что в этом ресторане предлагается только один тип кухни. Ответ округлите до одного знака после запятой.

In [20]:
df4 = main_task.copy()
df4.cuisine_style = df4.cuisine_style.str.replace("'", '')
df4.cuisine_style = df4.cuisine_style.str.replace("[", '')
df4.cuisine_style = df4.cuisine_style.str.replace("]", '')
df4.cuisine_style = df4.cuisine_style.str.replace(" ", '')
df4.cuisine_style = df4.cuisine_style.str.split(',')

def return_count(value):
    if type(value) == list:
        return len(value)
    else: return 1
df4['cuisine_number'] = df4.cuisine_style.apply(return_count)
round(df4.cuisine_number.mean(),1)

2.6

## 4.4 Вопросы об отзывах

Когда был оставлен самый свежий отзыв? Введите ответ в формате yyyy-mm-dd.


In [41]:
df = main_task.copy()
display(df.sample())
df.reviews.loc[0]


,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,URL_TA,ID_TA
31432,id_3524,Barcelona,"['Mediterranean', 'European', 'Spanish']",3525.0,3.5,$$$$,231.0,"[['Nice terrace', 'Good location and atmospher...",/Restaurant_Review-g187497-d1028624-Reviews-El...,d1028624


"[['Good food at your doorstep', 'A good hotel restaurant'], ['12/31/2017', '11/20/2017']]"

In [42]:
pattern = re.compile('\d+\W\d+\W\d\d\d\d')
df['reviews_date'] = df.reviews.apply(lambda x: pattern.findall(str(x)))
# df['reviews_date'] = df.reviews.str.replace(pattern, '')
df.reviews_date.loc[0]

['12/31/2017', '11/20/2017']

In [43]:
df.reviews_date.loc[8]

[]

In [44]:
r_date = pd.DataFrame(df.reviews_date.explode())
r_date.reviews_date = pd.to_datetime(r_date.reviews_date)

In [45]:
r_date.reviews_date.max()

Timestamp('2018-02-26 00:00:00')

Какое максимальное количество дней отделяет даты публикации отзывов, размещённых на сайте ресторана? Введите количество дней в виде целого числа.

In [46]:
first_review = r_date.groupby(r_date.index).min()
last_review = r_date.groupby(r_date.index).max()
delta_time = last_review - first_review
delta_time.max()

reviews_date   3207 days
dtype: timedelta64[ns]

In [ ]:
Добавим к датафрейму три переменные: время публикации последнего отзыва, время публиккации предпоследнего отзыва и разницу в днях между ними.

In [47]:
df['delta_reviews_time'] = delta_time.reviews_date
df['first_review_time'] = first_review.reviews_date
df['last_review_time'] = last_review.reviews_date

df.reviews.loc[df.delta_reviews_time == df.delta_reviews_time.max()]
# df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   restaurant_id       40000 non-null  object         
 1   city                40000 non-null  object         
 2   cuisine_style       30717 non-null  object         
 3   ranking             40000 non-null  float64        
 4   rating              40000 non-null  float64        
 5   price_range         26114 non-null  object         
 6   number_of_reviews   37457 non-null  float64        
 7   reviews             40000 non-null  object         
 8   URL_TA              40000 non-null  object         
 9   ID_TA               40000 non-null  object         
 10  reviews_date        40000 non-null  object         
 11  delta_reviews_time  33529 non-null  timedelta64[ns]
 12  first_review_time   33529 non-null  datetime64[ns] 
 13  last_review_time    33529 non-n

In [48]:
df[df.delta_reviews_time.isna()]

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,URL_TA,ID_TA,reviews_date,delta_reviews_time,first_review_time,last_review_time
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,[],NaT,NaT,NaT
8,id_2690,Vienna,NaN,2692.0,4.0,NaN,NaN,"[[], []]",/Restaurant_Review-g190454-d12845029-Reviews-G...,d12845029,[],NaT,NaT,NaT
10,id_6578,Barcelona,NaN,6579.0,3.0,NaN,6.0,"[[], []]",/Restaurant_Review-g187497-d10696479-Reviews-R...,d10696479,[],NaT,NaT,NaT
14,id_4773,Madrid,"['Mediterranean', 'Spanish', 'Healthy']",4776.0,4.0,$$ - $$$,7.0,"[[], []]",/Restaurant_Review-g187514-d10859423-Reviews-B...,d10859423,[],NaT,NaT,NaT
21,id_5844,Madrid,NaN,5847.0,4.0,NaN,NaN,"[[], []]",/Restaurant_Review-g187514-d10058810-Reviews-B...,d10058810,[],NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39981,id_924,Lyon,NaN,925.0,4.0,NaN,29.0,"[[], []]",/Restaurant_Review-g187265-d5850306-Reviews-Re...,d5850306,[],NaT,NaT,NaT
39986,id_7274,Madrid,NaN,7279.0,3.0,NaN,NaN,"[[], []]",/Restaurant_Review-g187514-d12182212-Reviews-C...,d12182212,[],NaT,NaT,NaT
39988,id_4968,Berlin,NaN,4970.0,4.0,NaN,NaN,"[[], []]",/Restaurant_Review-g187323-d7761701-Reviews-Ar...,d7761701,[],NaT,NaT,NaT
39991,id_6655,Paris,"['French', 'Contemporary', 'Fusion', 'Gastropub']",6656.0,5.0,$$ - $$$,5.0,"[[], []]",/Restaurant_Review-g187147-d13224611-Reviews-L...,d13224611,[],NaT,NaT,NaT


In [62]:
# df.delta_reviews_time = df.delta_reviews_time.datetime.timestamp()
# df.first_review_time = df.first_review_time.dt.timestamp()
# df.last_review_time = df.last_review_time.dt.timestamp() 
# datetime.timestamp() timestamp()
str(df.loc[0,'delta_reviews_time'])
# type(df.loc[4,'delta_reviews_time'])

'41 days 00:00:00'

Приведём переменную delta_reviews_time из типа pandas._libs.tslibs.timedeltas.Timedelta в int

In [89]:
df_ = df.copy()
df_.delta_reviews_time = df_.delta_reviews_time.apply(lambda x: str(x)).str.split(' ',expand=True)
df_.delta_reviews_time = df_.delta_reviews_time.replace('NaT',0)
df_.delta_reviews_time = df_.delta_reviews_time.astype(int)

df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   restaurant_id       40000 non-null  object        
 1   city                40000 non-null  object        
 2   cuisine_style       30717 non-null  object        
 3   ranking             40000 non-null  float64       
 4   rating              40000 non-null  float64       
 5   price_range         26114 non-null  object        
 6   number_of_reviews   37457 non-null  float64       
 7   reviews             40000 non-null  object        
 8   URL_TA              40000 non-null  object        
 9   ID_TA               40000 non-null  object        
 10  reviews_date        40000 non-null  object        
 11  delta_reviews_time  40000 non-null  int64         
 12  first_review_time   33529 non-null  datetime64[ns]
 13  last_review_time    33529 non-null  datetime64

In [92]:
df_.first_review_time.dt.day

0        20.0
1        19.0
2         6.0
3         NaN
4        19.0
         ... 
39995    12.0
39996    12.0
39997    12.0
39998    18.0
39999     4.0
Name: first_review_time, Length: 40000, dtype: float64


* Restaurant_id — идентификационный номер ресторана / сети ресторанов;
* City — город, в котором находится ресторан;
* Cuisine Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
* Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города;
* Rating — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
* Price Range — диапазон цен в ресторане;
* Number of Reviews — количество отзывов о ресторане;
* Reviews — данные о двух отзывах, которые отображаются на сайте ресторана;
* URL_TA — URL страницы ресторана на TripAdvisor;
* ID_TA — идентификатор ресторана в базе данных TripAdvisor.

